# Word Embeddings Generation

This Jupyter notebook demonstrates how to generate word embeddings from a given corpus using a pre-trained BERT model. These embeddings will be used to find semantically similar matches for a user query.

# Notebook Overview
- Import Dependencies
- Configure Logging and Define Constants and Paths
- Load and Preprocess Data
- Initialize BERT Tokenizer and Model
- Generate Embeddings in Batches
- Save Embeddings to File

# Import Dependencies

In [1]:
import sys
import os  
from pathlib import Path  
import logging
from datetime import datetime

# Data manipulation libraries
import pandas as pd
import numpy as np

# Deep learning framework
import torch  

# NLP libraries
import nltk  # Natural Language Toolkit
from nemo.collections.nlp.models import BERTLMModel  # BERT Language Model from NVIDIA NeMo
from transformers import AutoTokenizer  # Tokenizer for transformer-based models

In [2]:
# ------------------------ Suppress Verbose Logs ------------------------

import warnings
warnings.filterwarnings("ignore")

# Hugging Face Transformers logs
from transformers import logging as hf_logging
hf_logging.set_verbosity_error()

# NVIDIA NeMo logs
logging.getLogger("nemo_logger").setLevel(logging.ERROR)

# Configure Logging and Define Constants and Paths

In [3]:
# Create logger
logger = logging.getLogger("notebook_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S")  

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [4]:
CORPUS_PATH = "../data/raw/corpus.csv"
TOKENIZER_DIR = "../artifacts/tokenizer"
BERT_MODEL_NAME = "bert-large-uncased"
BERT_MODEL_DATAFABRIC_PATH = "/home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo"
EMBEDDINGS_OUTPUT_PATH = "../data/processed/"

In [5]:
logger.info('Notebook execution started.')

2025-04-07 02:17:57 - INFO - Notebook execution started.


# Load and Preprocess Data

In [6]:
# Download the Punkt tokenizer data for sentence tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
# Load the dataset into a Pandas DataFrame
corpus_df = pd.read_csv(CORPUS_PATH)

# Display the first few rows of the DataFrame
logger.info("First few entries of the DataFrame:")
print(corpus_df.head())

2025-04-07 02:17:57 - INFO - First few entries of the DataFrame:


   Unnamed: 0  Topic                                             Pledge
0           0      1  Actually we as an association are still pretty...
1           1      1  EFFAT welcomes the Commission Proposal for a R...
2           2      1  HOTREC calls for a level playing field and fai...
3           3      1  Estonia sees the need to synchronize and harmo...
4           4      1  Sphere Travel Club contributes to a flourishin...


In [8]:
documents = corpus_df["Pledge"].astype(str).tolist()  # Convert the column to a list

# Initialize BERT Tokenizer and Model

In [9]:
%%time

# Initialize the tokenizer with a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
tokenizer.save_pretrained(TOKENIZER_DIR)

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger.info("Loading BERT model...")

# Ensure you have added the 'bertlargeuncased' model from the NVIDIA NGC model catalog.
# If unavailable, use the alternative method below to download the model online.

# Uncomment the following line to download the BERT model online:
# bert_model = BERTLMModel.from_pretrained(model_name="bertlargeuncased", strict=False).to(device)

# Load the BERT model from a local .nemo file inside datafabric folder
bert_model = BERTLMModel.restore_from(BERT_MODEL_DATAFABRIC_PATH, strict=False).to(device)

logger.info("BERT model loaded successfully.")

2025-04-07 02:17:57 - INFO - Loading BERT model...
2025-04-07 02:19:44 - INFO - BERT model loaded successfully.


CPU times: user 15.2 s, sys: 4.37 s, total: 19.5 s
Wall time: 1min 47s


# Generate Embeddings in Batches

In [10]:
def generate_embeddings_in_batches(texts, tokenizer, model, batch_size=32):
    """
    Generates text embeddings using the NeMo BERT model in batches.
    
    Args:
        texts (list of str): List of input texts.
        tokenizer: Pretrained tokenizer.
        model: Pretrained NeMo BERT model.
        batch_size (int, optional): Batch size for processing. Default is 32.
    
    Returns:
        np.ndarray: Generated embeddings.
    """
    model.eval()  # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize batch with padding and truncation
        encoded_input = tokenizer(
            batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=128
        )
        encoded_input = {key: val.to(device) for key, val in encoded_input.items()}

        with torch.no_grad():  # Disable gradient computation for inference
            output = model.bert_model(**encoded_input)
        
        # Extract the CLS token representation for embeddings
        embeddings = output[:, 0, :].cpu().numpy()  # CLS token representation
        all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)

# Save Embeddings to File

In [11]:
%%time

# Generate embeddings using the pre-trained model
embeddings = generate_embeddings_in_batches(documents, tokenizer, bert_model)

# Convert embeddings into a DataFrame
df_embeddings = pd.DataFrame(embeddings)

# Ensure the output directory exists
os.makedirs(EMBEDDINGS_OUTPUT_PATH, exist_ok=True)
    
# Define output file path
output_file = os.path.join(EMBEDDINGS_OUTPUT_PATH, "embeddings.csv")

# Save embeddings
df_embeddings.to_csv(output_file , index=False)

logger.info(f"✅ Embedding completed and saved to: {output_file}")

2025-04-07 02:25:33 - INFO - ✅ Embedding completed and saved to: ../data/processed/embeddings.csv


CPU times: user 1h 27min 22s, sys: 2min 36s, total: 1h 29min 59s
Wall time: 5min 48s


In [12]:
logger.info('Notebook execution completed.')

2025-04-07 02:25:33 - INFO - Notebook execution completed.


Built with ❤️ using Z by HP AI Studio.